<a href="https://colab.research.google.com/github/sf01363/garbage_classification/blob/main/Efficientnet_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

batch_size = 16

train Loss: 0.0506 Acc: 0.9881
val Loss: 0.2090 Acc: 0.9331
Best val Acc: 0.945607
Test Acc: 0.9393

----------
combine
train Loss: 0.0531 Acc: 0.9854
val Loss: 0.1945 Acc: 0.9456
Best val Acc: 0.956067
Test Acc: 0.7301

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import os
import json

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

from PIL import Image
from pathlib import Path
import scipy

from torch.utils.data import Dataset
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
import pdb
from tqdm import tqdm

In [ ]:
import tensorflow as tf

if tf.test.gpu_device_name():
    print(f"GPU is available: {tf.test.gpu_device_name()}")
else:
    print("GPU is not available.")

GPU is available: /device:GPU:0


In [ ]:
!git clone https://github.com/PizaaRiaaa/garbage-classification.git

Cloning into 'garbage-classification'...
remote: Enumerating objects: 5148, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 5148 (delta 41), reused 49 (delta 17), pack-reused 5072
Receiving objects: 100% (5148/5148), 44.22 MiB | 11.78 MiB/s, done.
Resolving deltas: 100% (2181/2181), done.


In [ ]:
import os
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split, Subset

# 경로 설정
imagepath_cardboard = r"garbage-classification/Garbage/original_images/cardboard"
imagepath_glass = r"garbage-classification/Garbage/original_images/glass"
imagepath_metal = r"garbage-classification/Garbage/original_images/metal"
imagepath_paper = r"garbage-classification/Garbage/original_images/paper"
imagepath_plastic = r"garbage-classification/Garbage/original_images/plastic"
imagepath_trash = r"garbage-classification/Garbage/original_images/trash"

In [ ]:
# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 데이터셋 로드
data_dir = 'garbage-classification/Garbage/original_images'
dataset = datasets.ImageFolder(data_dir, transform=transform)


In [ ]:
# trash 데이터셋 로드 및 필터링
trash_indices = [i for i, (path, _) in enumerate(dataset.imgs) if 'trash' in path] # 클래스가 trash인 데이터의 인덱스
non_trash_indices = [i for i in range(len(dataset)) if i not in trash_indices] # 클래스가 trash가 아닌 데이터의 인덱스

trash_dataset = Subset(dataset, trash_indices)
non_trash_dataset = Subset(dataset, non_trash_indices)

In [ ]:
# 랜덤 시드 고정
import random
import torch

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [ ]:
# non_trash 데이터를 train, val, test로 분할
train_size = int(0.6 * len(non_trash_dataset))
val_size = int(0.2 * len(non_trash_dataset))
test_size = len(non_trash_dataset) - train_size - val_size

train_dataset, val_dataset, non_trash_test_dataset = random_split(non_trash_dataset, [train_size, val_size, test_size])

In [ ]:
# # non_trash 데이터를 train, val, test로 분할
# train_size = int(0.6 * len(non_trash_dataset))
# val_size = int(0.2 * len(non_trash_dataset))
# test_size = len(non_trash_dataset) - train_size - val_size

# train_dataset, val_dataset, non_trash_test_dataset = random_split(non_trash_dataset, [train_size, val_size, test_size])

# #테스트 데이터셋; non_trash_test_dataset + trash_dataset
# test_dataset = torch.utils.data.ConcatDataset([non_trash_test_dataset, trash_dataset])


In [ ]:
# 데이터 로더 설정

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(non_trash_test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# # 데이터 로더 설정

# batch_size = 16
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
from transformers import DeiTConfig, DeiTForImageClassification

##DeiT 모델 구조 관련 하이퍼파라미터
* Embedding Dimension (D): 입력 이미지 패치를 임베딩 벡터로 변환할 때의 차원 수입니다. 일반적으로 192, 384, 768 등으로 설정됩니다.
예: dim=384
* Patch Size (P): 입력 이미지를 나눌 패치의 크기입니다. 일반적으로 16x16, 32x32 등으로 설정됩니다.
예: patch_size=16
* Number of Layers (L): Transformer 블록의 층 수입니다. 모델의 깊이를 나타내며, 일반적으로 12, 24, 32 등으로 설정됩니다.
예: depth=12
* Number of Heads (H): Multi-head Self-Attention에서의 헤드 수입니다. 일반적으로 3, 6, 12 등으로 설정됩니다.
예: num_heads=6
* MLP Dimension (FFN): Transformer 블록 내의 Feed Forward Network(FFN)의 내부 차원입니다. 일반적으로 임베딩 차원의 2-4배로 설정됩니다.
예: mlp_dim=1536
* Dropout Rate: 드롭아웃 비율로, 과적합을 방지하기 위해 사용됩니다.
예: dropout=0.1
## 학습 관련 하이퍼파라미터
* Learning Rate (LR): 학습 속도를 조절하는 하이퍼파라미터로, 모델이 최적의 가중치를 찾는 속도에 영향을 미칩니다.
예: learning_rate=3e-4
* Batch Size: 한 번의 역전파에서 사용할 샘플 수입니다. 모델 학습의 효율성과 성능에 영향을 미칩니다.
예: batch_size=64
* Weight Decay: L2 정규화 항으로, 과적합을 방지하기 위해 사용됩니다.
예: weight_decay=0.05
* Number of Epochs: 전체 데이터셋을 몇 번 반복할지 결정하는 값입니다.
예: epochs=100
* Optimizer: 모델을 업데이트하는 데 사용되는 최적화 알고리즘입니다. AdamW가 일반적으로 사용됩니다.
예: optimizer=AdamW
* Warmup Steps: 학습 초기에 학습률을 점진적으로 증가시키는 단계 수입니다.
예: warmup_steps=5000

## DeiTConfig:
* image_size: 입력 이미지의 크기.
* patch_size: 입력 이미지를 나눌 패치 크기.
* num_channels: 입력 이미지의 채널 수 (일반적으로 RGB 이미지는 3).
* hidden_size: 임베딩 벡터의 크기.
* num_hidden_layers: Transformer 블록의 층 수.
* num_attention_heads: Multi-head Self-Attention에서의 헤드 수.
* intermediate_size: FFN의 내부 차원.
* dropout_rate: 드롭아웃 비율.
## Optimizer 및 Scheduler:
* AdamW: Adam의 변형으로, L2 정규화 대신 가중치 감쇠를 사용하여 더 나은 정규화 효과를 제공합니다.
* learning_rate=3e-4: 학습률.
* weight_decay=0.05: L2 정규화 강도.

In [ ]:
!pip install efficientnet_pytorch


  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━

In [ ]:
import torch
import torch.nn as nn
from efficientnet_pytorch import EfficientNet

class CustomEfficientNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomEfficientNet, self).__init__()

        # 사전 학습된 EfficientNet 모델 로드
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')

        # 기존 분류기 레이어의 특징 수 가져오기
        num_features = self.efficientnet._fc.in_features

        # 커스텀 분류기 정의
        self.efficientnet._fc = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.efficientnet(x)


In [ ]:
# 예제 사용법
num_classes = 5 # non_trash
model = CustomEfficientNet(num_classes=num_classes)

# # 예제 사용법
# num_classes = 6 # with trash
# model = CustomEfficientNet(num_classes=num_classes)

In [ ]:
# Move the model to the GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 옵티마이저 및 손실함수 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

# scheduler로 학습률 조정
from torch.optim.lr_scheduler import StepLR
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# 모델 학습
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Loaded pretrained weights for efficientnet-b0


CustomEfficientNet(
  (efficientnet): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d((0, 1, 0, 1))
    )
    (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
          (static_padding): ZeroPad2d((1, 1, 1, 1))
        )
        (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          32, 8, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          8, 32, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project_conv): Conv2dStati

In [ ]:
# 학습 함수
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=20):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # 각 에포크마다 훈련 및 검증
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 모델을 훈련 모드로 설정
                dataloader = train_loader
            else:
                model.eval()  # 모델을 검증 모드로 설정
                dataloader = val_loader

            running_loss = 0.0
            running_corrects = 0

            # 데이터 반복
            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # 모델을 깊은 복사(deep copy) 함
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    print('Best val Acc: {:4f}'.format(best_acc))

    # 최적의 모델 가중치 로드
    model.load_state_dict(best_model_wts)
    return model


In [ ]:
# 모델 학습
model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=25)

Epoch 0/24
----------
train Loss: 1.5952 Acc: 0.3808
val Loss: 1.0842 Acc: 0.7301
Epoch 1/24
----------
train Loss: 0.9876 Acc: 0.6771
val Loss: 0.6476 Acc: 0.8431
Epoch 2/24
----------
train Loss: 0.6724 Acc: 0.7978
val Loss: 0.4621 Acc: 0.8912
Epoch 3/24
----------
train Loss: 0.4855 Acc: 0.8724
val Loss: 0.3782 Acc: 0.9142
Epoch 4/24
----------
train Loss: 0.3710 Acc: 0.8933
val Loss: 0.3110 Acc: 0.9226
Epoch 5/24
----------
train Loss: 0.2939 Acc: 0.9331
val Loss: 0.2751 Acc: 0.9247
Epoch 6/24
----------
train Loss: 0.2388 Acc: 0.9484
val Loss: 0.2532 Acc: 0.9351
Epoch 7/24
----------
train Loss: 0.1962 Acc: 0.9561
val Loss: 0.2471 Acc: 0.9393
Epoch 8/24
----------
train Loss: 0.1653 Acc: 0.9665
val Loss: 0.2320 Acc: 0.9372
Epoch 9/24
----------
train Loss: 0.1426 Acc: 0.9714
val Loss: 0.2387 Acc: 0.9331
Epoch 10/24
----------
train Loss: 0.1452 Acc: 0.9686
val Loss: 0.2267 Acc: 0.9331
Epoch 11/24
----------
train Loss: 0.1221 Acc: 0.9749
val Loss: 0.2058 Acc: 0.9351
Epoch 12/24
--

In [ ]:
# 테스트 함수
def test_model(model, test_loader):
    model.eval()
    running_corrects = 0

    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

        running_corrects += torch.sum(preds == labels.data)

    test_acc = running_corrects.double() / len(test_loader.dataset)
    print(f'Test Acc: {test_acc:.4f}')

In [ ]:
# 모델 테스트
test_model(model, test_loader)

Test Acc: 0.7301


### 모델 저장

In [ ]:
# 모델 학습 후 저장하기
torch.save(model, 'model.pth')

# 저장된 모델 불러오기
#model = torch.load('model.pth')
#model = model.to(device)

# # 모델 학습 후 저장하기
# torch.save(model, 'model_combine.pth')

# # 저장된 모델 불러오기
# model = torch.load('model_combine.pth')
# model = model.to(device)

### 일반쓰레기 분류 정도 확인

In [ ]:
trash_dataset = Subset(dataset, trash_indices)
trash_loader = DataLoader(trash_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# 테스트 함수
def test_model_(model, test_loader):
    model.eval()
    running_corrects = 0

    # 전체 예측과 확률을 저장할 리스트
    all_preds = []
    all_probs = []
    max_probs = []

    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            probs = F.softmax(outputs, dim=1)  # 소프트맥스 함수로 확률값 계산

        running_corrects += torch.sum(preds == labels.data)

        # 예측과 확률 저장
        all_preds.extend(preds.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())
        max_probs.extend(torch.max(probs, dim=1)[0].cpu().numpy())


    test_acc = running_corrects.double() / len(test_loader.dataset)
    print(f'Test Acc: {test_acc:.4f}')

    # 각 이미지에 대한 예측 결과와 확률을 데이터프레임으로 생성
    df_data = {
        'Image Index': list(range(1, len(all_preds) + 1)),
        'Predicted Class': all_preds,
        'Probabilities': [', '.join([f'{prob:.5f}' for prob in prob_list]) for prob_list in all_probs],
         'Max Probability': [f'{prob:.5f}' for prob in max_probs]
    }
    df = pd.DataFrame(df_data)
    return df




In [ ]:
test_model_(model, trash_loader).head()

,Image Index,Predicted Class,Probabilities,Max Probability
0,1,4,"0.00575, 0.00901, 0.00828, 0.00761, 0.96935",0.96935
1,2,3,"0.08132, 0.00323, 0.00717, 0.89928, 0.00900",0.89928
2,3,2,"0.00591, 0.00153, 0.97254, 0.01722, 0.00280",0.97254
3,4,2,"0.02707, 0.01653, 0.57396, 0.03002, 0.35243",0.57396
4,5,2,"0.00616, 0.00366, 0.68365, 0.02086, 0.28568",0.68365
